[https://medium.com/@ab.jannatpour/advanced-pairs-trading-kalman-filters-b171144ce6d5](https://)

In [1]:
import sys
sys.path.append('../../')

In [2]:
from util.load_data import *
from config.constants import *

In [3]:
XAUUSD = load_data()

In [4]:
data_1 = XAUUSD['D1']

In [5]:
data_2 = pd.read_csv(
    PATH_DATA + '/USDJPY/USDJPY_D1 - USDJPY_D1' + '.csv', index_col='Time', parse_dates=True)

In [6]:
# Define the system model for the spread
A = 1
Q = 0.1

# Define the measurement model for the spread
H = 1
R = 0.1

In [7]:
mu = 0
sigma = 0.1

In [8]:
x = 0
P = 1

In [9]:
data_1

,Open,High,Low,Close,Volume
Time,,,,,
2007-01-01,635.679,637.103,635.006,635.943,604
2007-01-02,635.791,640.802,635.286,639.143,1282
2007-01-03,639.029,644.663,624.634,626.602,1302
2007-01-04,626.699,630.352,619.600,622.533,1365
2007-01-05,622.268,625.768,601.591,606.841,1284
...,...,...,...,...,...
2023-03-06,1852.255,1858.168,1845.015,1846.255,1380
2023-03-07,1846.225,1851.555,1812.655,1813.865,1380
2023-03-08,1813.855,1824.185,1809.248,1815.175,1379


In [10]:
data_2

,Open,High,Low,Close,Volume
Time,,,,,
2007-02-12,121.900,121.953,121.730,121.764,64849
2007-02-13,121.760,121.766,121.090,121.263,618320
2007-02-14,121.272,121.347,120.226,120.286,616539
2007-02-15,120.282,120.359,119.147,119.376,617115
2007-02-16,119.370,119.550,118.984,119.392,586044
...,...,...,...,...,...
2023-03-06,136.016,136.190,135.364,135.995,506836
2023-03-07,135.995,137.438,135.537,137.387,595894
2023-03-08,137.392,137.910,136.477,137.223,528313


In [11]:
data_1 = data_1.loc['2007-02-12':]

In [12]:
data_2 = data_2.loc[:'2023-03-10']

In [13]:
data = pd.concat([data_1, data_2], axis=1, keys=['StockA', 'StockB'])

In [14]:
data

StockA                                         StockB           \
                Open      High       Low     Close  Volume     Open     High   
Time                                                                           
2007-02-12   666.520   666.898   659.061   660.524  1329.0  121.900  121.953   
2007-02-13   660.511   668.870   659.846   663.033  1353.0  121.760  121.766   
2007-02-14   663.468   671.708   663.013   668.469  1333.0  121.272  121.347   
2007-02-15   668.441   670.400   661.906   667.930  1332.0  120.282  120.359   
2007-02-16   668.023   669.348   662.058   668.094  1187.0  119.370  119.550   
...              ...       ...       ...       ...     ...      ...      ...   
2023-03-06  1852.255  1858.168  1845.015  1846.255  1380.0  136.016  136.190   
2023-03-07  1846.225  1851.555  1812.655  1813.865  1380.0  135.995  137.438   
2023-03-08  1813.855  1824.185  1809.248  1815.175  1379.0  137.392  137.910   
2023-03-09  1815.195  1835.565  1811.805  1831.415  1366.0  137.223  137.254   
2023-03-10  1831.428  1867.348  1827.675  1864.098  1080.0  136.365  136.992   

                                        
                Low    Close    Volume  
Time                                    
2007-02-12  121.730  121.764   64849.0  
2007-02-13  121.090  121.263  618320.0  
2007-02-14  120.226  120.286  616539.0  
2007-02-15  119.147  119.376  617115.0  
2007-02-16  118.984  119.392  586044.0  
...             ...      ...       ...  
2023-03-06  135.364  135.995  506836.0  
2023-03-07  135.537  137.387  595894.0  
2023-03-08  136.477  137.223  528313.0  
2023-03-09  135.940  136.365  518978.0  
2023-03-10  134.111  134.738  579900.0  

[5030 rows x 10 columns]

In [15]:
data_1

,Open,High,Low,Close,Volume
Time,,,,,
2007-02-12,666.520,666.898,659.061,660.524,1329
2007-02-13,660.511,668.870,659.846,663.033,1353
2007-02-14,663.468,671.708,663.013,668.469,1333
2007-02-15,668.441,670.400,661.906,667.930,1332
2007-02-16,668.023,669.348,662.058,668.094,1187
...,...,...,...,...,...
2023-03-06,1852.255,1858.168,1845.015,1846.255,1380
2023-03-07,1846.225,1851.555,1812.655,1813.865,1380
2023-03-08,1813.855,1824.185,1809.248,1815.175,1379


In [16]:
data_2

,Open,High,Low,Close,Volume
Time,,,,,
2007-02-12,121.900,121.953,121.730,121.764,64849
2007-02-13,121.760,121.766,121.090,121.263,618320
2007-02-14,121.272,121.347,120.226,120.286,616539
2007-02-15,120.282,120.359,119.147,119.376,617115
2007-02-16,119.370,119.550,118.984,119.392,586044
...,...,...,...,...,...
2023-03-06,136.016,136.190,135.364,135.995,506836
2023-03-07,135.995,137.438,135.537,137.387,595894
2023-03-08,137.392,137.910,136.477,137.223,528313


In [17]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5007 entries, 2007-02-12 to 2023-03-10
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5007 non-null   float64
 1   High    5007 non-null   float64
 2   Low     5007 non-null   float64
 3   Close   5007 non-null   float64
 4   Volume  5007 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 363.7 KB


In [18]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5021 entries, 2007-02-12 to 2023-03-10
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5021 non-null   float64
 1   High    5021 non-null   float64
 2   Low     5021 non-null   float64
 3   Close   5021 non-null   float64
 4   Volume  5021 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 364.4 KB


In [44]:
import numpy as np
from pykalman import KalmanFilter
import pandas as pd

# defining a mean-reverting system model

# Assuming you have appropriate values for A, H, x, P, Q, and R
# ...
stock1 = data_1
stock2 = data_2

# calculate the spread
spread = stock1['Close'] - stock2['Close']

stock = spread.to_frame()
# Create the Kalman filter object



# Create a DataFrame to store signals
signals = pd.DataFrame(index=spread.index, columns=[
                       'stock1_signal', 'stock2_signal'])


# Signal generation logic

sigma = np.std(spread)

# Define the long-term mean
mu = np.mean(spread)

# Define the transition matrix
A = np.array([1])

# Define the transition covariance
Q = np.array([0.1])

# Define the measurement matrix
H = np.array([1])

# Define the measurement covariance
R = np.array([0.1])

# Define the initial state
x_init = np.array([mu])
P_init = np.array([1])


# Initialize the Kalman filter
kf = KalmanFilter(transition_matrices=A, observation_matrices=H,
                  transition_covariance=Q, observation_covariance=R,
                  initial_state_mean=x_init, initial_state_covariance=P_init)


# Loop over time steps
for i in range(len(spread)):
    # Prediction
    x_pred, P_pred = kf.filter_update(
        filtered_state_mean=x, filtered_state_covariance=P)

    # Measurement
    z = spread[i]

    # Update
    x, P = kf.filter_update(filtered_state_mean=x_pred,
                            filtered_state_covariance=P_pred, observation=z)
    
    (filtered_state_means, filtered_state_covariances) = kf.filter(spread[i])
  

   # Extract the final estimate of the spread
    estimated_spread = filtered_state_means[-1]
    # print(estimated_spread)

    # Extract the final estimate of the spread uncertainty
    estimated_spread_uncertainty = filtered_state_covariances[-1]
    # print(estimated_spread_uncertainty)

    # Signal generation logic


#    estimated_spread_uncertainty > sigma

    if estimated_spread > mu :
        signals.loc[spread.index[i], 'stock1_signal'] = 1
        signals.loc[spread.index[i], 'stock2_signal'] = -1
    elif estimated_spread < mu :
        signals.loc[spread.index[i], 'stock1_signal'] = -1
        signals.loc[spread.index[i], 'stock2_signal'] = 1
    else:
        signals.loc[spread.index[i], ['stock1_signal', 'stock2_signal']] = 0

In [45]:
signals.value_counts()

stock1_signal  stock2_signal
-1              1               2930
 1             -1               2068
 0              0                 32
dtype: int64

In [29]:
mu

1243.3694849939975

In [41]:
sigma

0.1

In [46]:
signals

,stock1_signal,stock2_signal
Time,,
2007-02-12,-1,1
2007-02-13,-1,1
2007-02-14,-1,1
2007-02-15,-1,1
2007-02-16,-1,1
...,...,...
2023-03-06,1,-1
2023-03-07,1,-1
2023-03-08,1,-1


In [47]:
signals.index[1]

Timestamp('2007-02-13 00:00:00')

In [49]:
data_1.loc[signals.index[1]]

Open       660.511
High       668.870
Low        659.846
Close      663.033
Volume    1353.000
Name: 2007-02-13 00:00:00, dtype: float64

In [51]:
data_1['returns'] = data_1['Close'] / data_1['Close'].shift(1)

,Open,High,Low,Close,Volume
Time,,,,,
2007-02-12,666.520,666.898,659.061,660.524,1329
2007-02-13,660.511,668.870,659.846,663.033,1353
2007-02-14,663.468,671.708,663.013,668.469,1333
2007-02-15,668.441,670.400,661.906,667.930,1332
2007-02-16,668.023,669.348,662.058,668.094,1187
...,...,...,...,...,...
2023-03-06,1852.255,1858.168,1845.015,1846.255,1380
2023-03-07,1846.225,1851.555,1812.655,1813.865,1380
2023-03-08,1813.855,1824.185,1809.248,1815.175,1379
